In [ ]:
import numpy as np

X_train = np.load('X_train.npy')
X_test = np.load('X_test.npy')

y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.15, random_state = 42)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
import keras as K

y_train = K.utils.np_utils.to_categorical(y_train)
y_test = K.utils.np_utils.to_categorical(y_test)
y_valid = K.utils.np_utils.to_categorical(y_valid)

In [ ]:
from keras.layers import Dense, Conv2D, Dropout, GRU, BatchNormalization, Input, Reshape, Activation, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import tensorflow as tf

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True # enable dynamic GPU memory allocation
session = tf.compat.v1.Session(config=config)

def SE_Block(input):
    # Squeeze
    glob_ave_pool = GlobalAveragePooling2D()(input)
    input_shape = input.shape[-1]

    # Excitation
    f = input_shape // 16

    fc1 = Dense(f, activation = 'relu', use_bias = False)(glob_ave_pool)
    fc2 = Dense(input_shape, activation = 'sigmoid', use_bias=False)(fc1)
    
    # Scaling
    re = Reshape((1, 1, input_shape))(fc2)

    output = input * re

    return output

def Double_GRU(input_shape):
    input = Input(input_shape)

    conv1 = Conv2D(32, (3,3))(input)
    batch1 = BatchNormalization()(conv1)
    act1 = Activation('relu')(batch1)
    se1 = SE_Block(act1)

    conv2 = Conv2D(32, (3,3))(se1)
    batch2 = BatchNormalization()(conv2)
    act2 = Activation('relu')(batch2)
    se2 = SE_Block(act2)

    conv3 = Conv2D(64, (3,3), strides=(2, 2))(se2)
    batch3 = BatchNormalization()(conv3)
    act3 = Activation('relu')(batch3)
    se3 = SE_Block(act3)

    conv4 = Conv2D(64, (3,3))(se3)
    batch4 = BatchNormalization()(conv4)
    act4 = Activation('relu')(batch4)
    se4 = SE_Block(act4)

    conv5 = Conv2D(128, (3,3))(se4)
    batch5 = BatchNormalization()(conv5)
    act5 = Activation('relu')(batch5)
    se5 = SE_Block(act5)

    reshape = Reshape((441, 128))(se5)
    dropout = Dropout(0.1)(reshape)

    gru1 = GRU(80, return_sequences = True)(dropout)
    gru2 = GRU(50)(gru1)

    reshape2 = Reshape((5, 10))(gru2)

    output = Dense(10, activation = 'softmax')(reshape2)

    model = Model(input, output, name = 'Double_gru')

    return model

In [ ]:
input_shape = (28, 140, 1)
model = Double_GRU(input_shape)
model.summary()

In [ ]:
import keras.backend  as K
def sequence_accuracy(y_true, y_pred):
    """
    Computes the proportion of sequences for which the entire sequence was correctly predicted.
    Assumes that each example in the batch has the same length.
    """
    # Compute the element-wise equality between y_true and y_pred
    equal_elements = K.cast(K.all(K.equal(y_true, K.round(y_pred)), axis=-1), K.floatx())

    # Compute the sequence-wise equality
    sequence_equal = K.all(equal_elements, axis=-1)

    # Compute the mean sequence accuracy across the batch
    sequence_accuracy = K.mean(sequence_equal)

    return sequence_accuracy

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience = 20,
    restore_best_weights=True
    )

model_hist = model.fit(
    x = X_train, y = y_train,
    epochs = 100,
    batch_size = 64,
    validation_data = (X_valid, y_valid),
    callbacks = [early_stopping]
)

In [ ]:
import matplotlib.pyplot as plt

plt.style.use("ggplot")
plt.figure(figsize=(10, 6))
plt.plot(model_hist.history["loss"])
plt.plot(model_hist.history["val_loss"])
plt.show()

In [ ]:
plt.style.use("ggplot")
plt.figure(figsize=(10, 6))
plt.plot(model_hist.history["accuracy"])
plt.plot(model_hist.history["val_accuracy"])
plt.show()

In [ ]:
score = model.evaluate(X_test, y_test)

print('Model accuracy : ', score[1])
print('Model loss : ', score[0])

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
import tensorflow as tf

pred_label = tf.argmax(y_pred, axis = -1)
pred_label = np.array(pred_label)

label = tf.argmax(y_test, axis = -1)
label = np.array(label)

In [ ]:
y_test_flat = np.ravel(pred_label)
y_pred_flat = np.ravel(label)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

confusion_matrix = confusion_matrix(y_test_flat, y_pred_flat)

print(classification_report(y_test_flat, y_pred_flat))
print(accuracy_score(y_test_flat, y_pred_flat))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

ax = plt.subplots(figsize=(10, 6))
ax = sns.heatmap(confusion_matrix, annot=True, cmap='Blues', fmt="d")
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values')

ax.xaxis.set_ticklabels(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])
ax.yaxis.set_ticklabels(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])

plt.show()

In [ ]:
# Find the maximum value in each row
max_values = np.max(y_pred, axis=2)

# Create a new array with 1 where the maximum value is found, and 0 elsewhere
result = np.where(y_pred == max_values[..., np.newaxis], 1, 0)

In [ ]:
a = sequence_accuracy(y_test, result)
a